In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandasql as psql

# read in csv file into dataframe using pandas
df_team = pd.read_csv("../input/nbaplayoffseasonaldata/team_season.txt",header=0)
df_player_regular = pd.read_csv("../input/nbaplayoffseasonaldata/player_regular_season.txt",header=0) 
df_player_playoff = pd.read_csv("../input/nbaplayoffseasonaldata/player_playoffs.txt",header=0)
df_player_allstar = pd.read_csv("../input/nbaplayoffseasonaldata/player_allstar.txt",header=0)

df_team_info = pd.read_csv("../input/nbaplayoffseasonaldata/teams.txt")
y = pd.read_csv("/kaggle/input/new-ylabel/new_y.csv")

In [ ]:
# get required 2002-2003 seasonal data
df_team_02_03 = df_team.loc[(df_team.year >= 2002) & (df_team.year <= 2003)]
df_team_02_03.loc[:,['team', 'year', 'o_pts', 'd_pts', 'won', 'lost']]
df_team_02_03_avg = psql.sqldf("select team, AVG(o_pts) as AVG_opts, AVG(d_pts) as AVG_dpts, avg(won) as AVG_won, AVG(lost) as AVG_lost from df_team_02_03 group by team")
df_team_02_03_avg
y
y_train = y.iloc[:,1:-1].mean(axis=1)
y_pred = y.iloc[:,-1]


In [ ]:
df_team_03_04 = df_team.loc[df_team.year == 2004]
df_team_03_04.loc[:,['team', 'year', 'o_pts', 'd_pts', 'won', 'lost']]
df_team_03_04_avg = psql.sqldf("select team, AVG(o_pts) as AVG_opts, AVG(d_pts) as AVG_dpts, avg(won) as AVG_won, AVG(lost) as AVG_lost from df_team_03_04 group by team")
df_team_03_04_avg

In [ ]:
# all star players in year 2002 and 2003
df_allstar = df_player_allstar.loc[(df_player_allstar.year >= 2002) & (df_player_allstar.year <= 2003)]
df_allstar.head(5)

In [ ]:
Allstars = psql.sqldf("select DISTINCT ilkid, firstname, lastname from df_allstar group by lastname;")
print(Allstars)

In [ ]:
Player_team = psql.sqldf("select ilkid, year, firstname, lastname, team from df_player_playoff where year >= 2002 AND year<= 2003;")
Player_team

In [ ]:

df_allstar_team = psql.sqldf("select distinct P.ilkid, P.firstname, P.lastname, P.team from Player_team as P JOIN Allstars as A ON UPPER(A.ilkid) = UPPER(P.ilkid) and P.lastname = A.lastname;")

NumberOfAllstarPlayer = psql.sqldf("select team, COUNT(team) as NumberOfAllstarPlayer from df_allstar_team group by team")


allstar_sum = psql.sqldf("select N.team, T.name, N.NumberOfAllstarPlayer from NumberOfAllstarPlayer as N INNER JOIN team_fullname as T on N.team = T.team")

In [ ]:
X = psql.sqldf("select a.*, NumberOfAllStarPlayer from df_team_02_03_avg as a left join allstar_sum as b on a.team=b.team")
X = X.fillna(0)
X

In [ ]:
X_test = psql.sqldf("select a.*, NumberOfAllStarPlayer from df_team_03_04_avg as a left join allstar_sum as b on a.team=b.team")
X_test = X_test.fillna(0)
X_test

In [ ]:
regular = psql.sqldf("select * from df_player_regular where year >= 2002 AND year<= 2003;")
playoff = psql.sqldf("select * from df_player_playoff where year >= 2002 AND year<= 2003;")

# efficiency calculation formula from https://en.wikipedia.org/wiki/Efficiency_(basketball)

regular_efficiency = psql.sqldf("SELECT team, ilkid, firstname, lastname, pts, (pts+oreb+dreb+asts+stl+blk-turnover-fgm-ftm)/gp as efficiency from regular group by ilkid order by efficiency desc")
playoff_efficiency = psql.sqldf("SELECT team, ilkid, firstname, lastname, pts, (pts+oreb+dreb+asts+stl+blk-turnover-fgm-ftm)/gp as efficiency from playoff group by ilkid order by efficiency desc")

In [ ]:
test_regular = psql.sqldf("select * from df_player_regular where year == 2004;")
test_playoff = psql.sqldf("select * from df_player_playoff where year == 2004;")

# efficiency calculation formula from https://en.wikipedia.org/wiki/Efficiency_(basketball)

test_regular_efficiency = psql.sqldf("SELECT team, ilkid, firstname, lastname, pts, (pts+oreb+dreb+asts+stl+blk-turnover-fgm-ftm)/gp as efficiency from test_regular group by ilkid order by efficiency desc")
test_playoff_efficiency = psql.sqldf("SELECT team, ilkid, firstname, lastname, pts, (pts+oreb+dreb+asts+stl+blk-turnover-fgm-ftm)/gp as efficiency from test_playoff group by ilkid order by efficiency desc")

In [ ]:
total_efficiency = psql.sqldf("Select P.team, P.ilkid, P.firstname, P.lastname, (P.pts*0.7 + R.pts*0.3) as weighted_pts, (P.efficiency*0.7 + R.efficiency*0.3) as weighted_efficiency from regular_efficiency as R INNER JOIN playoff_efficiency as P on P.firstname=R.firstname and P.lastname=R.lastname order by weighted_efficiency desc limit 100;")

total_efficiency.head(20)

In [ ]:
test_total_efficiency = psql.sqldf("Select P.team, P.ilkid, P.firstname, P.lastname, (P.pts*0.7 + R.pts*0.3) as weighted_pts, (P.efficiency*0.7 + R.efficiency*0.3) as weighted_efficiency from test_regular_efficiency as R INNER JOIN test_playoff_efficiency as P on P.firstname=R.firstname and P.lastname=R.lastname order by weighted_efficiency desc limit 100;")

test_total_efficiency.head(20)

In [ ]:
pts = psql.sqldf("select weighted_pts, weighted_efficiency from total_efficiency;")

In [ ]:
# normalization of value

from sklearn import preprocessing

x = pts.values #returns a numpy array 

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)



In [ ]:
total_efficiency = psql.sqldf("Select P.team, P.ilkid, P.firstname, P.lastname, (P.pts*0.7 + R.pts*0.3) as weighted_pts, (P.efficiency*0.7 + R.efficiency*0.3) as weighted_efficiency from regular_efficiency as R INNER JOIN playoff_efficiency as P on P.firstname=R.firstname and P.lastname=R.lastname order by weighted_efficiency desc;")

X = psql.sqldf("select X.*, avg(T.weighted_pts) as AVG_weighted_pts, avg(T.weighted_efficiency) as AVG_weighted_efficiency from X left join total_efficiency as T on X.team = T.team GROUP BY X.team")
X_train = X.fillna(0)

X_train.iloc[:,:]

In [ ]:
X_test = psql.sqldf("select X_test.*, avg(T.weighted_pts) as AVG_weighted_pts, avg(T.weighted_efficiency) as AVG_weighted_efficiency from X_test left join total_efficiency as T on X_test.team = T.team GROUP BY X_test.team")
X_test = X_test.fillna(0)
X_test_scale = X_test.iloc[:,1:]
X_test.iloc[:,:]

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
X_test_scale = min_max_scaler.fit_transform(X_test_scale)

In [ ]:
# Import
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
poly = PolynomialFeatures(degree=1)
X_ = poly.fit_transform(X_train.iloc[:,1:])
X_test = poly.fit_transform(X_test.iloc[:,1:])


In [ ]:
# Instantiate
lg = LinearRegression()

# Fit
lg.fit(X_, y_train)

y_pred = lg.predict(X_test)

np.argsort(-1 * y_pred)
